In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QSlider, QSpacerItem, QSizePolicy
from PyQt5.QtCore import Qt
from mpl_toolkits.mplot3d import Axes3D

# Constants
mu_0 = 4 * np.pi * 1e-7  # Permeability of free space (T·m/A)
I = 10  # Current in Amperes

# Create a 3D Grid
x_vals = np.linspace(-2, 2, 20)
y_vals = np.linspace(-2, 2, 20)
z_vals = np.linspace(-2, 2, 20)
X, Y, Z = np.meshgrid(x_vals, y_vals, z_vals)

# Compute distances from X-axis and Y-axis
R_x = np.sqrt(Y**2 + Z**2)  
R_y = np.sqrt(X**2 + Z**2)  

# Magnetic Field components from wire along X-axis (circular in YZ plane)
Bx_x = np.zeros_like(X)  
By_x = - (mu_0 * I / (2 * np.pi * R_x)) * (Z / R_x)  
Bz_x = (mu_0 * I / (2 * np.pi * R_x)) * (Y / R_x)  

# Magnetic Field components from wire along Y-axis (circular in XZ plane)
Bx_y = (mu_0 * I / (2 * np.pi * R_y)) * (Z / R_y)  
By_y = np.zeros_like(Y)  
Bz_y = - (mu_0 * I / (2 * np.pi * R_y)) * (X / R_y)  

# Total field components
Bx = Bx_x + Bx_y
By = By_x + By_y
Bz = Bz_x + Bz_y

# Avoid NaN values at the center
Bx[np.isnan(Bx)] = 0
By[np.isnan(By)] = 0
Bz[np.isnan(Bz)] = 0

# Compute Magnetic Field Magnitude
B_magnitude = np.sqrt(Bx**2 + By**2 + Bz**2)

# Set fixed color scale limits
vmin, vmax = np.min(B_magnitude), np.max(B_magnitude)


class MagneticFieldGUI(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Magnetic Field Visualization")
        self.showMaximized()  # Open in full-screen mode

        self.main_widget = QWidget(self)
        self.setCentralWidget(self.main_widget)
        self.layout = QVBoxLayout(self.main_widget)

        # Layout for plots
        self.plot_layout = QHBoxLayout()
        self.layout.addLayout(self.plot_layout)

        # 3D Heatmap Figure
        self.figure_3d, self.ax_3d = plt.subplots(subplot_kw={"projection": "3d"})
        self.canvas_3d = FigureCanvas(self.figure_3d)
        self.canvas_3d.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)
        self.plot_layout.addWidget(self.canvas_3d)

        self.plot_3d_heatmap()
        
        # Connect scroll event for zooming in Z-direction
        self.canvas_3d.mpl_connect("scroll_event", self.on_scroll)

        # 2D Contour Figure
        self.figure_2d, self.ax_2d = plt.subplots()
        self.canvas_2d = FigureCanvas(self.figure_2d)
        self.canvas_2d.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)
        self.plot_layout.addWidget(self.canvas_2d)

        # Horizontal layout for sliders
        self.slider_layout = QHBoxLayout()
        self.layout.addLayout(self.slider_layout)

        # Add sliders with flexible width
        self.slider_x = self.create_slider("X Plane", self.plot_2d_contour, len(x_vals) // 2, len(x_vals))
        self.slider_y = self.create_slider("Y Plane", self.plot_2d_contour, len(y_vals) // 2, len(y_vals))
        self.slider_z = self.create_slider("Z Plane", self.plot_2d_contour, len(z_vals) // 2, len(z_vals))

        self.plot_2d_contour()

    def plot_3d_heatmap(self):
        """Plots a fully interactive 3D heatmap inside the PyQt5 window."""
        self.ax_3d.clear()
        X_flat, Y_flat, Z_flat, B_flat = X.flatten(), Y.flatten(), Z.flatten(), B_magnitude.flatten()

        scatter = self.ax_3d.scatter(X_flat, Y_flat, Z_flat, c=B_flat, cmap="plasma", alpha=0.9)
        self.figure_3d.colorbar(scatter, ax=self.ax_3d, label="Field Strength (T)")

        self.ax_3d.set_xlabel("X-axis")
        self.ax_3d.set_ylabel("Y-axis")
        self.ax_3d.set_zlabel("Z-axis")
        self.ax_3d.set_title("3D Heatmap of Magnetic Field")

        self.ax_3d.set_zlim(-2, 2)  # Initial Z limit
        self.canvas_3d.draw()

    def on_scroll(self, event):
        """Handles zooming in and out on the Z-axis when scrolling."""
        z_min, z_max = self.ax_3d.get_zlim()
        zoom_factor = 0.1  # Adjust zoom speed

        if event.step > 0:  # Scroll Up → Expand Z-axis
            new_z_min = z_min - zoom_factor
            new_z_max = z_max + zoom_factor
        else:  # Scroll Down → Contract Z-axis
            new_z_min = z_min + zoom_factor
            new_z_max = z_max - zoom_factor

        self.ax_3d.set_zlim(new_z_min, new_z_max)
        self.canvas_3d.draw()



    def create_slider(self, label_text, callback, default, max_value):
        slider_widget = QWidget()
        slider_layout = QVBoxLayout(slider_widget)
        
        label = QLabel(label_text, self)
        slider = QSlider(Qt.Horizontal)
        slider.setMinimum(0)
        slider.setMaximum(max_value - 1)
        slider.setValue(default)
        slider.setMinimumWidth(200)  # Prevents sliders from being too small
        slider.setMaximumWidth(500)  # Prevents sliders from being too big
        slider.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Fixed)  # Allows resizing proportionally
        slider.valueChanged.connect(callback)

        slider_layout.addWidget(label)
        slider_layout.addWidget(slider)
        self.slider_layout.addWidget(slider_widget)

        return slider

    def plot_2d_contour(self):
        """Updates the 2D contour plot based on the selected plane with a fixed scale."""
        self.ax_2d.clear()
        x_index = self.slider_x.value()
        y_index = self.slider_y.value()
        z_index = self.slider_z.value()

        # Contour plot for the selected plane
        contour = self.ax_2d.contourf(X[:, :, z_index], Y[:, :, z_index], B_magnitude[:, :, z_index], 
                                      cmap="plasma", vmin=vmin, vmax=vmax)
        contour_lines = self.ax_2d.contour(X[:, :, z_index], Y[:, :, z_index], B_magnitude[:, :, z_index], 
                                           colors='black', linewidths=0.5)
        self.ax_2d.clabel(contour_lines, inline=True, fontsize=8)

        # Keep the colorbar constant
        if not hasattr(self, "colorbar"):
            self.colorbar = self.figure_2d.colorbar(contour, ax=self.ax_2d, label="Field Strength (T)")
        else:
            self.colorbar.update_normal(contour)

        self.ax_2d.set_xlabel("X-axis")
        self.ax_2d.set_ylabel("Y-axis")
        self.ax_2d.set_title(f"Contour Plot at Z={z_vals[z_index]:.2f}")

        self.canvas_2d.draw()


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MagneticFieldGUI()
    window.show()
    sys.exit(app.exec_())
